In [ ]:
import MySQLdb
from pandas.io import sql
import pandas as pd
import numpy as np
conn = MySQLdb.connect(host="localhost", user="root", passwd="MYSQLROOTPASSWORD", db="openmrs") 
openmrscursor = conn.cursor() 


In [ ]:
df_concept = pd.read_sql('select  * from `concept`', con = conn)
df_concept

In [ ]:
df_conceptname = pd.read_sql('select * from `concept_name`', con = conn) 
len(df_conceptname.index)

In [ ]:
nan_in_col  = df_concept[df_concept['description'].isna()]
nan_in_col

In [ ]:
df_concept = df_concept[df_concept['description'].notna()]
df_concept

In [ ]:
df_concept.loc[df_concept['short_name'] == 'LOS']

# d_icd_diagnoses to concept

In [ ]:
mimicconn = MySQLdb.connect(host="localhost", user="root", passwd="MYSQLROOTPASSWORD", db="mimic4") 
mimiccursor = mimicconn.cursor()
concept_d = pd.read_sql('select * from d_icd_diagnoses', con = mimicconn) 
concept_d

In [ ]:
concept_dx_mimic=pd.read_sql('select long_title, any_value (icd_code) from mimic4.d_icd_diagnoses where icd_code IN (SELECT icd_code FROM mimic4.admissions adm, mimic4.diagnoses_icd diag where adm.hadm_id=diag.hadm_id and diag.seq_num=1) group by (long_title)', con=mimicconn)
concept_dx_mimic

In [ ]:
# concept_dx_mimic_copy = concept_dx_mimic.drop(columns=['icd_version'])
concept_dx_mimic = concept_dx_mimic.rename(columns={"any_value (icd_code)": "short_name", "long_title": "description"})
concept_dx_mimic['date_created'] = '2022-01-01'
concept_dx_mimic

In [ ]:
# concept_dx_mimic_copy.to_csv('concept_dx_mimic_copy.csv', index = False)

In [ ]:
import uuid
concept_dx_mimic['uuid'] = [uuid.uuid4() for x in range(len(concept_dx_mimic))]

In [ ]:
import pymysql

# Connect to the database
connection = pymysql.connect(host='localhost',
                         user='root',
                         password='Soic@2022Capstone',
                         db='openmrs')

# create cursor
cursor=connection.cursor() 

In [ ]:
column_names = ['short_name', 'description', 'date_created', 'uuid']

concept_dx_mimic = concept_dx_mimic.reindex(columns=column_names)
concept_dx_mimic

In [ ]:
# creating column list for insertion for concept table
cols = "`,`".join(['short_name', 'description', 'date_created', 'uuid'])

# Insert DataFrame recrds one by one.
for i,row in concept_dx_mimic.iterrows():
    sql1 = "SET FOREIGN_KEY_CHECKS=0"
    sql = "INSERT INTO `concept` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    sql2 = "SET FOREIGN_KEY_CHECKS=1"
    try:
        cursor.execute(sql1)
        cursor.execute(sql, tuple(row))
        cursor.execute(sql2)
        
    except MySQLdb.OperationalError:
        continue

    # the connection is not autocommitted by default, so we must commit to save our changes
    connection.commit() 

datatype_id = 'UPDATE concept SET datatype_id=4'
class_id = 'UPDATE concept SET class_id=4'
cursor.execute(datatype_id)
cursor.execute(class_id)
connection.commit() 

# d_icd_procedures to concept 

In [ ]:
concept_proc_mimic = pd.read_sql('select * from d_icd_procedures', con = mimicconn) 
concept_proc_mimic

In [ ]:
concept_proc_mimic['long_title'].nunique()

In [ ]:
concept_proc_mimic[concept_proc_mimic['icd_code']== '031']

In [ ]:
a = concept_proc_mimic[concept_proc_mimic['icd_version']==9]

In [ ]:
b = concept_proc_mimic[concept_proc_mimic['icd_version']==10]

In [ ]:
y = set(b['icd_code'])

In [ ]:
concept_proc_mimic_copy = concept_proc_mimic.drop(columns=['icd_version'])
concept_proc_mimic_copy = concept_proc_mimic_copy.rename(columns={"icd_code": "short_name", "long_title": "description"})

In [ ]:
concept_proc_mimic_copy['date_created'] = '2022-01-01'
concept_proc_mimic_copy['uuid'] = [uuid.uuid4() for x in range(len(concept_proc_mimic_copy))]

In [ ]:
concept_proc_mimic_copy

In [ ]:
# creating column list for insertion for concept table
cols = "`,`".join(['short_name', 'description', 'date_created', 'uuid'])

# Insert DataFrame recrds one by one.
for i,row in concept_proc_mimic_copy.iterrows():
    sql1 = "SET FOREIGN_KEY_CHECKS=0"
    sql = "INSERT INTO `concept` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    sql2 = "SET FOREIGN_KEY_CHECKS=1"
    try:
        cursor.execute(sql1)
        cursor.execute(sql, tuple(row))
        cursor.execute(sql2)
        
    except MySQLdb.OperationalError:
        continue

    # the connection is not autocommitted by default, so we must commit to save our changes
    connection.commit() 

In [ ]:
datatype_id = 'UPDATE concept SET datatype_id=4'
class_id = 'UPDATE concept SET class_id=2 where class_id!=4'
concept_creator = 'UPDATE concept SET creator = 1' 

cursor.execute(datatype_id)
cursor.execute(class_id)
cursor.execute(concept_creator)


connection.commit() 

# Importing into concept_name from concept

In [ ]:
concept_name_mimic = "INSERT INTO concept_name(concept_id, name, date_created, locale, creator, concept_name_type, uuid) select concept_id, description, date_created,'en', 1, 'FULLY_SPECIFIED', uuid() from concept"
openmrscursor.execute(concept_name_mimic)
conn.commit() 

# creating concept for LOS

In [ ]:
concept_los = "INSERT INTO concept(short_name,description,datatype_id,class_id,creator,date_created,uuid) VALUES ('LOS', 'Length of Stay', 1, 5, 1, '2022-01-01', uuid())"
cursor.execute(concept_los)
connection.commit()

In [ ]:
sql1 = "SET FOREIGN_KEY_CHECKS=0"
concept_name_los = "INSERT INTO concept_name(concept_id, name, date_created, locale, creator, concept_name_type, uuid) VALUES (1179194, 'Length of Stay', '2022-01-01', 'en', 1, 'FULLY_SPECIFIED', uuid())" 
sql2 = "SET FOREIGN_KEY_CHECKS=1"
cursor.execute(sql1)
cursor.execute(concept_name_los)
cursor.execute(sql2)
connection.commit()

In [ ]:
#"INSERT INTO visit(indication_concept_id) SELECT concept_id from concept where openmrs.concept(short_name) = mimic4.diagnoses_icd(icd_code) AND openmrs.visit(visit_id) = mimic4.diagnoses_icd(hadm_id)"

In [ ]:
# sql1 = "SET FOREIGN_KEY_CHECKS=0"
sql2 = "Delete from concept_name"
# sql3 = "SET FOREIGN_KEY_CHECKS=1"
# cursor.execute(sql1)
cursor.execute(sql2)
# cursor.execute(sql3)
connection.commit() 

In [ ]:
# inserting LOS concept and LOS value into concept_numeric

In [ ]:
df_concept_numeric=pd.read_sql('select * from `concept_numeric`', con = conn) 
df_concept_numeric

In [ ]:
df_icustays_mimic = pd.read_sql('select * from icustays', con = mimicconn) 
df_icustays_mimic

In [ ]:
df_icustays_mimic['concept_id'] = 1482681
df_icustays_mimic

In [ ]:
concept_numeric_los = "INSERT INTO concept_numeric(concept_id) VALUES (1482681)"
openmrscursor.execute(concept_numeric_los)
conn.commit()

In [ ]:
openmrscursor.execute('UPDATE concept_numeric SET allow_decimal = 1')
conn.commit()